<a href="https://colab.research.google.com/github/Vaibhav-S75/Team_404_Object_Detection/blob/main/WEEK_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from pycocotools.coco import COCO
from PIL import Image
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Install pycocotools if not already installed
!pip install pycocotools

# Define paths - update these to match your Google Drive structure
ANNOTATION_FILE = '/content/drive/MyDrive/annotations/annotations/instances_train2014.json'
IMAGE_DIR = '/content/drive/MyDrive/train2014/train2014'

def load_and_preprocess_image(image_path):
    try:
        image = Image.open(image_path)
        image = image.convert('RGB')  # Ensure 3 color channels
        image = image.resize((224, 224))
        return np.array(image) / 255.0
    except Exception as e:
        print(f"Error loading image {image_path}: {str(e)}")
        return None

# Load COCO annotations
print("Loading COCO annotations...")
coco = COCO(ANNOTATION_FILE)

# Get all image ids
image_ids = coco.getImgIds()

# Load a subset of images (e.g., 1000 images)
num_images = 1000
print(f"Loading {num_images} images...")

images = []
categories = []

for img_id in image_ids[:num_images]:
    img_info = coco.loadImgs(img_id)[0]
    image_path = os.path.join(IMAGE_DIR, img_info['file_name'])

    img = load_and_preprocess_image(image_path)
    ann_ids = coco.getAnnIds(imgIds=img_id)

    if img is not None and ann_ids:
        images.append(img)
        ann = coco.loadAnns(ann_ids[0])[0]
        categories.append(ann['category_id'])
    else:
        if img is None:
            print(f"Error loading image: {image_path}")
        if not ann_ids:
            print(f"No annotations found for image ID: {img_id}")

# Convert to NumPy arrays
images = np.array(images)
categories = np.array(categories)

print(f"Loaded {len(images)} images successfully.")
print(f"Loaded {len(categories)} categories successfully.")

# Ensure the number of images and categories match
if len(images) != len(categories):
    raise ValueError("Mismatch between number of images and categories.")

# Convert to one-hot encoding
num_classes = len(coco.getCatIds())
max_category_id = np.max(categories)
num_classes = max(num_classes, max_category_id + 1)  # Add 1 for 0-indexing
categories_one_hot = tf.keras.utils.to_categorical(categories, num_classes=num_classes)

# Split the data into training and validation sets
split = int(0.8 * len(images))
train_images, val_images = images[:split], images[split:]
train_labels, val_labels = categories_one_hot[:split], categories_one_hot[split:]

# Validate shapes before training
print(f"Training images shape: {train_images.shape}")
print(f"Training labels shape: {train_labels.shape}")

# Define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
print("Training the model...")
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(val_images, val_labels))

# Evaluate the model
print("Evaluating the model...")
test_loss, test_acc = model.evaluate(val_images, val_labels, verbose=2)
print(f'\nTest accuracy: {test_acc}')

# Save the model
model.save('/content/drive/MyDrive/coco_cnn_model.h5')
print("Model saved to Google Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading COCO annotations...
loading annotations into memory...
Done (t=27.72s)
creating index...
index created!
Loading 1000 images...
Error loading image /content/drive/MyDrive/train2014/train2014/COCO_train2014_000000057870.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/train2014/train2014/COCO_train2014_000000057870.jpg'
Error loading image: /content/drive/MyDrive/train2014/train2014/COCO_train2014_000000057870.jpg
Error loading image /content/drive/MyDrive/train2014/train2014/COCO_train2014_000000222016.jpg: [Errno 2] No such file or directory: '/content/drive/MyDrive/train2014/train2014/COCO_train2014_000000222016.jpg'
Error loading image: /content/drive/MyDrive/train2014/train2014/COCO_train2014_000000222016.jpg
Error loading image /content/drive/MyDrive/train2014/train2014/COCO_train2014_000000520950.jpg: [Errno 2] No such file or di

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training the model...
Epoch 1/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 46s 3s/step - accuracy: 0.0858 - loss: 5.1454 - val_accuracy: 0.2233 - val_loss: 4.4556
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.1793 - loss: 3.9819 - val_accuracy: 0.2233 - val_loss: 4.2690
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - accuracy: 0.1630 - loss: 3.8250 - val_accuracy: 0.1942 - val_loss: 4.4995
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 78s 3s/step - accuracy: 0.2320 - loss: 3.3906 - val_accuracy: 0.1748 - val_loss: 4.3734
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.2439 - loss: 2.9826 - val_accuracy: 0.0971 - val_loss: 5.0868
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.4525 - loss: 2.1206 - val_accuracy: 0.0680 - val_loss: 6.4777
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.7280 - loss: 1.1290 - val_accuracy: 0.0291 - val_loss: 7.8359
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 82s 3s/step - accuracy: 0.9267 - loss: 0.4220 - val_accura


Test accuracy: 0.09708737581968307
Model saved to Google Drive.
